Setup PyTorch to use best hardware option

In [1]:
import torch

torch.backends.cuda.matmul.allow_tf32 = True

if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(device)

mps


In [2]:
from datasets import load_dataset

dataset = load_dataset("jayavibhav/prompt-injection")
test_dataset = dataset['test']
test_dataset = test_dataset.rename_column("text", "prompt")

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "protectai/deberta-v3-base-prompt-injection"
tokenizer = AutoTokenizer.from_pretrained(model_name, clean_up_tokenization_spaces=False, use_fast=True, max_length=512)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)
model.eval()

DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine

In [4]:
def tokenize_batch(batch):
    return tokenizer(batch["prompt"], padding='max_length', truncation=True, max_length=512)

In [5]:
prompts_train_tokenized = test_dataset.map(tokenize_batch, batched=True)

Map:   0%|          | 0/65416 [00:00<?, ? examples/s]

In [6]:
from tqdm import tqdm

batch_size = 8
predictions = []
total_batches = len(prompts_train_tokenized) // batch_size + (1 if len(prompts_train_tokenized) % batch_size != 0 else 0)

with torch.no_grad():
    progress_bar = tqdm(total=total_batches, desc="Running inference", unit="batch")
    for batch in prompts_train_tokenized.select_columns(["input_ids", "attention_mask"]).with_format("torch").iter(batch_size=batch_size):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        predictions.extend(outputs.logits.argmax(dim=-1).cpu().tolist())
        progress_bar.update(1)
    
    progress_bar.close()

Running inference: 100%|██████████| 8177/8177 [2:25:06<00:00,  1.06s/batch]  


In [7]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

true_labels = prompts_train_tokenized["label"]
accuracy = accuracy_score(true_labels, predictions)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='binary')

In [8]:
import pandas as pd

metrics_df = pd.DataFrame({
    'Accuracy': [accuracy],
    'Precision': [precision],
    'Recall': [recall],
    'F1 Score': [f1]
})

# Format all columns to display 4 decimal places
for column in metrics_df.columns:
    metrics_df[column] = metrics_df[column].apply(lambda x: f"{x:.4f}")

metrics_df

,Accuracy,Precision,Recall,F1 Score
0,0.5571,0.8044,0.1359,0.2325
